In [3]:
import sys
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from nbeats_pytorch.model import NBeatsNet
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim

sys.path.append('/home/../multiTS/NFT/')
from dicts import data_to_num_vars_dict
from models.training_functions import read_all_data_and_print_stats, train_model, evaluate_model, save_model
from models.baseline_models.base_models import FC, LSTM, CNN, TCN, TimeSeriesTransformer

data = 'seasonality'

NUM_OF_VARS = data_to_num_vars_dict[data]
LOOKBACK = 5
HORIZON = 1

BATCH_SIZE = 32
NUM_EPOCHS = 1
PLOT_EPOCH = NUM_EPOCHS

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# save_model_path = f"NFT/models/baseline_models/{data}/"

n_people = 600
n_rows = 700
    
class dataset(Dataset):
    def __init__(self, data, target):
        self.input = data
        self.target = target

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx): 
        return self.input[idx], self.target[idx]


In [ ]:
if data == 'ecg':
    data_path = f"NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h_{n_people}people_{n_rows}rows/"
else:
    data_path = f"NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h/"

train_X, train_y, val_X, val_y, test_X, test_y = read_all_data_and_print_stats(data_path=data_path)


In [ ]:
def evaluate_model(model, train_X, train_y, val_X, val_y, test_X, test_y):
    def print_evaluation(train_mse, val_mse, test_mse,
                        train_smape, val_smape, test_smape, 
                        train_mape, val_mape, test_mape,
                        train_mase, val_mase, test_mase):
                                    
        print(f"MSE:\n"
        f"train MSE = {train_mse}\n"
        f"val MSE = {val_mse}\n"
        f"test MSE = {test_mse}\n")
        
        print(f"sMAPE:\n"
        f"train sMAPE = {train_smape}\n"
        f"val sMAPE = {val_smape}\n"
        f"test sMAPE = {test_smape}\n")
        
        print(f"MAPE:\n"
        f"train MAPE = {train_mape}\n"
        f"val MAPE = {val_mape}\n"
        f"test MAPE = {test_mape}\n")
        
        print(f"MASE:\n"
        f"train MAPE = {train_mase}\n"
        f"val MAPE = {val_mase}\n"
        f"test MAPE = {test_mase}\n")
    
    model.eval()

    # if isinstance(train_X, np.ndarray): train_X = torch.from_numpy(train_X)
    # if isinstance(val_X, np.ndarray): train_X = torch.from_numpy(val_X)
    # if isinstance(test_X, np.ndarray): train_X = torch.from_numpy(test_X)

    train_pred = model.predict(train_X)
    val_pred = model.predict(val_X)
    test_pred = model.predict(test_X)
    
    if isinstance(train_pred, np.ndarray): train_pred = torch.from_numpy(train_pred)
    if isinstance(val_pred, np.ndarray): val_pred = torch.from_numpy(val_pred)
    if isinstance(test_pred, np.ndarray): test_pred = torch.from_numpy(test_pred)

    train_mse = F.mse_loss(train_pred.to(device), train_y.to(device))
    val_mse = F.mse_loss(val_pred.to(device), val_y.to(device))
    test_mse = F.mse_loss(test_pred.to(device), test_y.to(device))

    train_smape = calculate_smape(train_pred, train_y)
    val_smape = calculate_smape(val_pred, val_y)
    test_smape = calculate_smape(test_pred, test_y)
    
    train_mape = calculate_mape(train_pred, train_y)
    val_mape = calculate_mape(val_pred, val_y)
    test_mape = calculate_mape(test_pred, test_y)
    
    train_mase = calculate_mase(train_pred, train_y)
    val_mase = calculate_mase(val_pred, val_y)
    test_mase = calculate_mase(test_pred, test_y)
    
    print_evaluation(train_mse, val_mse, test_mse,
                     train_smape, val_smape, test_smape, 
                     train_mape, val_mape, test_mape,
                     train_mase, val_mase, test_mase)
    
    return train_pred, val_pred, test_pred, train_mse, test_mse, train_smape, test_smape, train_mape, test_mape, train_mase, test_mase



In [ ]:
tcn_model = TCN(lookback=LOOKBACK, 
                horizon=HORIZON, 
                num_vars=NUM_OF_VARS, 
                device=device,
                num_channels=[25, 50], 
                kernel_size=2, 
                dropout=0.2).to(device)

train_model(dataset=dataset,
            epochs=NUM_EPOCHS,
            model=tcn_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None
            )

evaluate_model(tcn_model, train_X, train_y, val_X, val_y, test_X, test_y)

# save_model(tcn_model, "tcn", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
import pickle
import random
from time import time
from typing import Union

import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.nn.functional import mse_loss, l1_loss, binary_cross_entropy, cross_entropy
from torch.optim import Optimizer


class NBeatsNet(nn.Module):
    SEASONALITY_BLOCK = 'seasonality'
    TREND_BLOCK = 'trend'
    GENERIC_BLOCK = 'generic'

    def __init__(
            self,
            stack_types=(TREND_BLOCK, SEASONALITY_BLOCK),
            nb_blocks_per_stack=3,
            forecast_length=5,
            backcast_length=10,
            thetas_dim=(4, 8),
            share_weights_in_stack=False,
            hidden_layer_units=256,
            nb_harmonics=None
    ):
        super(NBeatsNet, self).__init__()
        self.forecast_length = forecast_length
        self.backcast_length = backcast_length
        self.hidden_layer_units = hidden_layer_units
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.share_weights_in_stack = share_weights_in_stack
        self.nb_harmonics = nb_harmonics
        self.stack_types = stack_types
        self.stacks = []
        self.thetas_dim = thetas_dim
        self.parameters = []
        print('| N-Beats')
        for stack_id in range(len(self.stack_types)):
            self.stacks.append(self.create_stack(stack_id))
        self.parameters = nn.ParameterList(self.parameters)
        self._loss = None
        self._opt = None
        self._gen_intermediate_outputs = False
        self._intermediary_outputs = []

    def create_stack(self, stack_id):
        stack_type = self.stack_types[stack_id]
        print(f'| --  Stack {stack_type.title()} (#{stack_id}) (share_weights_in_stack={self.share_weights_in_stack})')
        blocks = []
        for block_id in range(self.nb_blocks_per_stack):
            block_init = NBeatsNet.select_block(stack_type)
            if self.share_weights_in_stack and block_id != 0:
                block = blocks[-1]  # pick up the last one when we share weights.
            else:
                block = block_init(
                    self.hidden_layer_units, self.thetas_dim[stack_id],
                    self.device, self.backcast_length, self.forecast_length,
                    self.nb_harmonics
                )
                self.parameters.extend(block.parameters())
            print(f'     | -- {block}')
            blocks.append(block)
        return blocks

    def disable_intermediate_outputs(self):
        self._gen_intermediate_outputs = False

    def enable_intermediate_outputs(self):
        self._gen_intermediate_outputs = True

    def save(self, filename: str):
        torch.save(self, filename)

    @staticmethod
    def load(f, map_location=None, pickle_module=pickle, **pickle_load_args):
        return torch.load(f, map_location, pickle_module, **pickle_load_args)

    @staticmethod
    def select_block(block_type):
        if block_type == NBeatsNet.SEASONALITY_BLOCK:
            return SeasonalityBlock
        elif block_type == NBeatsNet.TREND_BLOCK:
            return TrendBlock
        else:
            return GenericBlock

    def compile(self, loss: str, optimizer: Union[str, Optimizer]):
        if loss == 'mae':
            loss_ = l1_loss
        elif loss == 'mse':
            loss_ = mse_loss
        elif loss == 'cross_entropy':
            loss_ = cross_entropy
        elif loss == 'binary_crossentropy':
            loss_ = binary_cross_entropy
        else:
            raise ValueError(f'Unknown loss name: {loss}.')
        # noinspection PyArgumentList
        if isinstance(optimizer, str):
            if optimizer == 'adam':
                opt_ = optim.Adam
            elif optimizer == 'sgd':
                opt_ = optim.SGD
            elif optimizer == 'rmsprop':
                opt_ = optim.RMSprop
            else:
                raise ValueError(f'Unknown opt name: {optimizer}.')
            opt_ = opt_(lr=1e-4, params=self.parameters())
        else:
            opt_ = optimizer
        self._opt = opt_
        self._loss = loss_

    def fit(self, x_train, y_train, validation_data=None, epochs=10, batch_size=32):

        def split(arr, size):
            arrays = []
            while len(arr) > size:
                slice_ = arr[:size]
                arrays.append(slice_)
                arr = arr[size:]
            arrays.append(arr)
            return arrays

        for epoch in range(epochs):
            x_train_list = split(x_train, batch_size)
            y_train_list = split(y_train, batch_size)
            assert len(x_train_list) == len(y_train_list)
            shuffled_indices = list(range(len(x_train_list)))
            random.shuffle(shuffled_indices)
            self.train()
            train_loss = []
            timer = time()
            for batch_id in shuffled_indices:
                batch_x, batch_y = x_train_list[batch_id], y_train_list[batch_id]
                self._opt.zero_grad()
                _, forecast = self(batch_x.clone().detach().to(self.device))
                loss = self._loss(forecast, squeeze_last_dim(batch_y.clone().detach().to(self.device)))
                train_loss.append(loss.item())
                loss.backward()
                self._opt.step()
            elapsed_time = time() - timer
            train_loss = np.mean(train_loss)

            test_loss = '[undefined]'
            if validation_data is not None:
                x_test, y_test = validation_data
                self.eval()
                _, forecast = self(x_test.clone().detach().to(self.device))
                test_loss = self._loss(forecast, squeeze_last_dim(y_test.clone().detach())).item()

            num_samples = len(x_train_list)
            time_per_step = int(elapsed_time / num_samples * 1000)
            print(f'Epoch {str(epoch + 1).zfill(len(str(epochs)))}/{epochs}')
            print(f'{num_samples}/{num_samples} [==============================] - ')
            print(f'{int(elapsed_time)}s {time_per_step}ms/step - ')
            # print(f'loss: {train_loss:.4f} - val_loss: {test_loss:.4f}')
            
    def predict(self, x, return_backcast=False):
        self.eval()
        b, f = self(x.clone().detach().to(self.device))
        b, f = b.detach().cpu().numpy(), f.detach().cpu().numpy()
        if len(x.shape) == 3:
            b = np.expand_dims(b, axis=-1)
            f = np.expand_dims(f, axis=-1)
        if return_backcast:
            return b
        return f

    @staticmethod
    def name():
        return 'NBeatsPytorch'

    def get_generic_and_interpretable_outputs(self):
        g_pred = sum([a['value'][0] for a in self._intermediary_outputs if 'generic' in a['layer'].lower()])
        i_pred = sum([a['value'][0] for a in self._intermediary_outputs if 'generic' not in a['layer'].lower()])
        outputs = {o['layer']: o['value'][0] for o in self._intermediary_outputs}
        return g_pred, i_pred, outputs

    def forward(self, backcast):
        self._intermediary_outputs = []
        backcast = squeeze_last_dim(backcast)
        forecast = torch.zeros(size=(backcast.size()[0], self.forecast_length,))  # maybe batch size here.
        for stack_id in range(len(self.stacks)):
            for block_id in range(len(self.stacks[stack_id])):
                b, f = self.stacks[stack_id][block_id](backcast)
                backcast = backcast.to(self.device) - b
                forecast = forecast.to(self.device) + f
                block_type = self.stacks[stack_id][block_id].__class__.__name__
                layer_name = f'stack_{stack_id}-{block_type}_{block_id}'
                if self._gen_intermediate_outputs:
                    self._intermediary_outputs.append({'value': f.detach().numpy(), 'layer': layer_name})
        return backcast, forecast


def squeeze_last_dim(tensor):
    if len(tensor.shape) == 3 and tensor.shape[-1] == 1:  # (128, 10, 1) => (128, 10).
        return tensor[..., 0]
    return tensor


def seasonality_model(thetas, t, device):
    p = thetas.size()[-1]
    assert p <= thetas.shape[1], 'thetas_dim is too big.'
    p1, p2 = (p // 2, p // 2) if p % 2 == 0 else (p // 2, p // 2 + 1)
    s1 = torch.tensor(np.array([np.cos(2 * np.pi * i * t) for i in range(p1)])).float()  # H/2-1
    s2 = torch.tensor(np.array([np.sin(2 * np.pi * i * t) for i in range(p2)])).float()
    S = torch.cat([s1, s2])
    return thetas.mm(S.to(device))


def trend_model(thetas, t, device):
    p = thetas.size()[-1]
    assert p <= 4, 'thetas_dim is too big.'
    T = torch.tensor(np.array([t ** i for i in range(p)])).float()
    return thetas.mm(T.to(device))


def linear_space(backcast_length, forecast_length, is_forecast=True):
    horizon = forecast_length if is_forecast else backcast_length
    return np.arange(0, horizon) / horizon


class Block(nn.Module):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, share_thetas=False,
                 nb_harmonics=None):
        super(Block, self).__init__()
        self.units = units
        self.thetas_dim = thetas_dim
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.share_thetas = share_thetas
        self.fc1 = nn.Linear(backcast_length, units)
        self.fc2 = nn.Linear(units, units)
        self.fc3 = nn.Linear(units, units)
        self.fc4 = nn.Linear(units, units)
        self.device = device
        self.backcast_linspace = linear_space(backcast_length, forecast_length, is_forecast=False)
        self.forecast_linspace = linear_space(backcast_length, forecast_length, is_forecast=True)
        if share_thetas:
            self.theta_f_fc = self.theta_b_fc = nn.Linear(units, thetas_dim, bias=False)
        else:
            self.theta_b_fc = nn.Linear(units, thetas_dim, bias=False)
            self.theta_f_fc = nn.Linear(units, thetas_dim, bias=False)

    def forward(self, x):
        x = squeeze_last_dim(x)
        x = F.relu(self.fc1(x.to(self.device)))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

    def __str__(self):
        block_type = type(self).__name__
        return f'{block_type}(units={self.units}, thetas_dim={self.thetas_dim}, ' \
               f'backcast_length={self.backcast_length}, forecast_length={self.forecast_length}, ' \
               f'share_thetas={self.share_thetas}) at @{id(self)}'


class SeasonalityBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        if nb_harmonics:
            super(SeasonalityBlock, self).__init__(units, nb_harmonics, device, backcast_length,
                                                   forecast_length, share_thetas=True)
        else:
            super(SeasonalityBlock, self).__init__(units, forecast_length, device, backcast_length,
                                                   forecast_length, share_thetas=True)

    def forward(self, x):
        x = super(SeasonalityBlock, self).forward(x)
        backcast = seasonality_model(self.theta_b_fc(x), self.backcast_linspace, self.device)
        forecast = seasonality_model(self.theta_f_fc(x), self.forecast_linspace, self.device)
        return backcast, forecast


class TrendBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        super(TrendBlock, self).__init__(units, thetas_dim, device, backcast_length,
                                         forecast_length, share_thetas=True)

    def forward(self, x):
        x = super(TrendBlock, self).forward(x)
        backcast = trend_model(self.theta_b_fc(x), self.backcast_linspace, self.device)
        forecast = trend_model(self.theta_f_fc(x), self.forecast_linspace, self.device)
        return backcast, forecast


class GenericBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        super(GenericBlock, self).__init__(units, thetas_dim, device, backcast_length, forecast_length)

        self.backcast_fc = nn.Linear(thetas_dim, backcast_length)
        self.forecast_fc = nn.Linear(thetas_dim, forecast_length)

    def forward(self, x):
        # no constraint for generic arch.
        x = super(GenericBlock, self).forward(x)

        theta_b = self.theta_b_fc(x)
        theta_f = self.theta_f_fc(x)

        backcast = self.backcast_fc(theta_b)  # generic. 3.3.
        forecast = self.forecast_fc(theta_f)  # generic. 3.3.

        return backcast, forecast


In [ ]:
nbeats_model = NBeatsNet(
        stack_types=(NBeatsNet.TREND_BLOCK, NBeatsNet.SEASONALITY_BLOCK),
        forecast_length=HORIZON,
        backcast_length=LOOKBACK,
        nb_blocks_per_stack=2,
        thetas_dim=(4,8),
        device=device,
        ).to(device)

for idx in range(NUM_OF_VARS):
        print(f"idx={idx}")
        print(f"train_X={train_X.shape}")

        train_X_idx, train_y_idx = train_X[:,:,idx], train_y[:,:,idx]
        val_X_idx, val_y_idx = val_X[:,:,idx], val_y[:,:,idx]
        test_X_idx, test_y_idx = test_X[:,:,idx], test_y[:,:,idx]
        print(f"train_X={train_X_idx.shape}")
        train_model(
                dataset=dataset, 
                epochs=NUM_EPOCHS, 
                model=nbeats_model, 
                train_X=train_X_idx.to(device), train_y=train_y_idx.to(device),
                val_X=val_X_idx.to(device), val_y=val_y_idx.to(device), 
                device=device, 
                lookback=LOOKBACK,
                horizon=HORIZON, 
                n_vars=NUM_OF_VARS, 
                batch_size=BATCH_SIZE, 
                print_epoch=PLOT_EPOCH, 
                path_to_save_prediction_plots=None)
        
        evaluate_model(nbeats_model, train_X_idx, train_y_idx, val_X_idx, val_y_idx, test_X_idx, test_y_idx)


In [ ]:
fc_model = FC(input_dim=NUM_OF_VARS, 
              output_dim=NUM_OF_VARS,
              horizon=HORIZON, 
              ).to(device)

train_model(dataset=dataset, 
            epochs=NUM_EPOCHS, 
            model=fc_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None)

evaluate_model(fc_model, train_X.to(device), train_y.to(device), val_X.to(device), val_y.to(device), test_X, test_y)

# save_model(fc_model, "fc", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
lstm_model = LSTM(num_vars=NUM_OF_VARS, 
                     lookback=LOOKBACK, 
                     horizon=HORIZON, 
                     hidden_dim=50, 
                     num_layers=2, 
                     device=device).to(device)

train_model(dataset=dataset, 
            epochs=NUM_EPOCHS, 
            model=lstm_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None)

evaluate_model(lstm_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(lstm_model, "lstm", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
cnn_model = CNN(lookback=LOOKBACK, 
                horizon=HORIZON, 
                num_vars=NUM_OF_VARS,
                device=device,
                num_channels=[16], 
                kernel_size=2, 
                dropout=0.2).to(device)

train_model(dataset=dataset,
            epochs=NUM_EPOCHS,
            model=cnn_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None
            )

evaluate_model(cnn_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(cnn_model, "cnn", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
transformer_model = TimeSeriesTransformer(
    lookback=LOOKBACK,
    horizon=HORIZON,
    num_vars=NUM_OF_VARS,
    device=device,
    num_layers=1,
    num_heads=1,
    dim_feedforward=16,
).to(device)

train_model(dataset=dataset, 
            epochs=NUM_EPOCHS, 
            model=transformer_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None)

evaluate_model(transformer_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(transformer_model, "transformer", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)